# Simple Autoencoder

In this notebook we will create a simple autoencoder that just learns a simple system represented by a linear function.

We will use a simple Dense network with Stochastic Gradient Descent optimizer. 

## Importing the Libraries.

We begin by importing our libraries:

In [9]:
#Simple Autoencoder

from keras.layers import Dense, Input
from keras.models import Model
from keras.optimizers import SGD
from sklearn.metrics import mean_squared_error
import numpy as np

# For generating our fake data
import random

We now set some hyperparameters, including # of epochs to train, # of datapoints, as well as noise and # of testing data.

Note that we have DATA_NOISE set at approximately 10% of our maximum and minimum data values, and TEST_NOISE set at 30%.

In [2]:
# Number of epochs to train
EPOCHS = 200

# Number of fake datapoints to create
                                                                                                               
DATA_SIZE = 10000
DATA_NOISE = 0.1
VAL_SIZE = int(0.2 * DATA_SIZE)

# Test Noise level
TEST_NOISE = 0.3

## Creating our Simple Autoencoder

We will now create our autoencoder. We use a fully connected dense network with 3 hidden layers with 32, 16 and 32 hidden nodes. We use ReLu activation in all hidden layers and tanh for the output layer, which allows us to create and learn data from \[-1, 1\]

In [10]:
# We use the Functional API to create our autoencoder
aeinput = Input(shape = (2, ), name = 'input')
encoder = Dense(units = 32, activation = 'relu')(aeinput)
encoder = Dense(units = 16, activation = 'relu')(encoder) 
decoder = Dense(units = 32, activation = 'relu')(encoder)
aeoutput = Dense(units = 2, activation = 'tanh')(decoder)

ae = Model(aeinput, aeoutput)
ae.compile(loss = 'mean_squared_error', optimizer = 'sgd')


## Generating the Data

Let's now generate the data! We set a fixed random seed (24601.. haha), then create our random datapoints for X between 0 and 1. We then produce our Y data which goes from -1 to 1, and add some synthetic noise.

Finally we call np.array to turn everything into NumPy arrays, which is needed by Keras.

In [11]:
# We set a fixed seed so that our experiments are reproducible
random.seed(24601)

# This adds white noise of -scale to scale
def noise(scale):
    return (2 * random.uniform(0, 1) - 1) * scale 

def gen_X(data_size):
    return [random.uniform(0, 1) for i in range(data_size)]

def gen_Y(X):
    return [[x, (2*x + noise(DATA_NOISE) - 1)] for x in X]
# We create our dummy dataset and output, with 1000 numbers
X_in = gen_X(DATA_SIZE)
X_test = gen_X(VAL_SIZE)

# We create a simple linear function with noise for the network to learn
Y = gen_Y(X_in)
Y_out = np.array(Y)
Y_test = np.array(gen_Y(X_test))

## Creating Noisy and Wrong Data

We now create our very noisy data, as well as data that is produced using the wrong function ($1.5 \times x$ instead of $2 \times x$), so that we can evaluate how our simple autoencoder behaves.

In [12]:
# We create a noisy version of Y_out to check the anomaly detection
Y_noisy = np.array([[x, 2 * x + noise(TEST_NOISE) - 1] for x in X_in])

# We create an incorrect version of Y_out
Y_wrong = np.array([[x, 1.5*x + noise(DATA_NOISE) - 1]for x in X_in])


## Training and Evaluation

Finally let's train our autoencoder:

In [13]:
ae.fit(x = Y_out, y = Y_out, batch_size = 100, 
epochs = EPOCHS, validation_data = (Y_test, Y_test))

clean_loss = ae.evaluate(x = Y_out, y = Y_out)
test_loss = ae.evaluate(x = Y_test, y = Y_test)
noisy_loss = ae.evaluate(x = Y_noisy, y = Y_noisy)
wrong_loss = ae.evaluate(x = Y_wrong, y = Y_wrong)

print("Clean loss = %3.4f, Test loss = %3.4f Noisy loss = %3.4f, Wrong loss = %3.4f" % 
(clean_loss, test_loss, noisy_loss, wrong_loss))

Train on 10000 samples, validate on 2000 samples
Epoch 1/200
10000/10000 [==============================] - 0s 25us/step - loss: 0.2311 - val_loss: 0.1482
Epoch 2/200
10000/10000 [==============================] - 0s 10us/step - loss: 0.1103 - val_loss: 0.0734
Epoch 3/200
10000/10000 [==============================] - 0s 9us/step - loss: 0.0520 - val_loss: 0.0330
Epoch 4/200
10000/10000 [==============================] - 0s 10us/step - loss: 0.0238 - val_loss: 0.0164
Epoch 5/200
10000/10000 [==============================] - 0s 9us/step - loss: 0.0131 - val_loss: 0.0106
Epoch 6/200
10000/10000 [==============================] - 0s 10us/step - loss: 0.0093 - val_loss: 0.0083
Epoch 7/200
10000/10000 [==============================] - 0s 9us/step - loss: 0.0077 - val_loss: 0.0073
Epoch 8/200
10000/10000 [==============================] - 0s 15us/step - loss: 0.0068 - val_loss: 0.0066
Epoch 9/200
10000/10000 [==============================] - 0s 15us/step - loss: 0.0063 - val_loss: 0.0062


10000/10000 [==============================] - 0s 10us/step - loss: 0.0016 - val_loss: 0.0016
Epoch 78/200
10000/10000 [==============================] - ETA: 0s - loss: 0.001 - 0s 10us/step - loss: 0.0015 - val_loss: 0.0016
Epoch 79/200
10000/10000 [==============================] - 0s 10us/step - loss: 0.0015 - val_loss: 0.0016
Epoch 80/200
10000/10000 [==============================] - 0s 11us/step - loss: 0.0015 - val_loss: 0.0015
Epoch 81/200
10000/10000 [==============================] - 0s 10us/step - loss: 0.0015 - val_loss: 0.0015
Epoch 82/200
10000/10000 [==============================] - 0s 11us/step - loss: 0.0015 - val_loss: 0.0015
Epoch 83/200
10000/10000 [==============================] - 0s 10us/step - loss: 0.0015 - val_loss: 0.0015
Epoch 84/200
10000/10000 [==============================] - 0s 11us/step - loss: 0.0015 - val_loss: 0.0015
Epoch 85/200
10000/10000 [==============================] - 0s 10us/step - loss: 0.0015 - val_loss: 0.0015
Epoch 86/200
10000/10000 [

Epoch 153/200
10000/10000 [==============================] - 0s 11us/step - loss: 0.0011 - val_loss: 0.0011
Epoch 154/200
10000/10000 [==============================] - 0s 10us/step - loss: 0.0011 - val_loss: 0.0011
Epoch 155/200
10000/10000 [==============================] - 0s 10us/step - loss: 0.0011 - val_loss: 0.0011
Epoch 156/200
10000/10000 [==============================] - 0s 10us/step - loss: 0.0011 - val_loss: 0.0011
Epoch 157/200
10000/10000 [==============================] - 0s 10us/step - loss: 0.0011 - val_loss: 0.0011
Epoch 158/200
10000/10000 [==============================] - 0s 11us/step - loss: 0.0011 - val_loss: 0.0011
Epoch 159/200
10000/10000 [==============================] - 0s 11us/step - loss: 0.0011 - val_loss: 0.0011
Epoch 160/200
10000/10000 [==============================] - 0s 12us/step - loss: 0.0011 - val_loss: 0.0011
Epoch 161/200
10000/10000 [==============================] - 0s 11us/step - loss: 0.0011 - val_loss: 0.0011
Epoch 162/200
10000/10000 [=

Now let's evaluate the losses for the:

    - Training data
    - Test data
    - Very noisy test data
    - Incorrect data
    
From here we can see how the autoencoder behaves for each type of test:

In [14]:
clean_loss = ae.evaluate(x = Y_out, y = Y_out)
test_loss = ae.evaluate(x = Y_test, y = Y_test)
noisy_loss = ae.evaluate(x = Y_noisy, y = Y_noisy)
wrong_loss = ae.evaluate(x = Y_wrong, y = Y_wrong)

print("\n\nClean loss = %3.4f, Test loss = %3.4f Noisy loss = %3.4f, Wrong loss = %3.4f" % 
(clean_loss, test_loss, noisy_loss, wrong_loss))


10000/10000 [==============================] - 0s 13us/step


Clean loss = 0.0010, Test loss = 0.0010 Noisy loss = 0.0043, Wrong loss = 0.0070


## Analysis

From the results above, we can see that the test loss is at 0.0009, while the noisy loss and wrong loss are a whole order of magnitude larger at 0.0040 and 0.0060. This allows us to set a threshold of perhaps 0.002 to determine when readings from a system (simulated here by a simple linear system) deviates too far from its normal operation.